In [14]:
%%time

import os.path
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from tqdm.dask import TqdmCallback

from shared_code.utility.spark.set_environ import set_azure_env

cb = TqdmCallback(desc="global")
cb.register()

tqdm.pandas()
tqdm.pandas(desc="global")

pbar = ProgressBar()
pbar.register()

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter

fs_adapter = AzureFileStorageAdapter('data')
file_system = fs_adapter.get_file_storage()

CPU times: total: 5.08 s
Wall time: 8.34 s
CPU times: total: 31.2 ms
Wall time: 134 ms


In [15]:
from typing import List
from dataclasses import dataclass


@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)

In [16]:
def add_source(x: object, source_list) -> str:
	for source in source_list:
		if x['subreddit'] in source['data']:
			return source['name']
	return ""

In [17]:
def add_compressed_data(x: object) -> str:
	try:
		data = open(x['path'], 'rb').read()
		return data
	except Exception as e:
		print(e)
		return ""

In [18]:
%%time
all_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(all_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,a close up of a white swan with a black and wh...,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,D:\data\images\itookapicture\thumbnail\b9d9119...,True,False
21395,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,a woman in a black dress holding a skull headd...,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,D:\data\images\itookapicture\thumbnail\17fc708...,True,False
21396,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,a close up of a bald eagle sitting on a tree b...,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,D:\data\images\itookapicture\thumbnail\a7205f6...,True,False
21397,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,there are two flamingos standing in the grass ...,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,D:\data\images\itookapicture\thumbnail\827260f...,True,False


CPU times: total: 734 ms
Wall time: 1.65 s


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,a close up of a white swan with a black and wh...,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,D:\data\images\itookapicture\thumbnail\b9d9119...,True,False
21395,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,a woman in a black dress holding a skull headd...,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,D:\data\images\itookapicture\thumbnail\17fc708...,True,False
21396,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,a close up of a bald eagle sitting on a tree b...,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,D:\data\images\itookapicture\thumbnail\a7205f6...,True,False
21397,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,there are two flamingos standing in the grass ...,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,D:\data\images\itookapicture\thumbnail\827260f...,True,False


CPU times: total: 766 ms
Wall time: 1.53 s


In [19]:
%%time

filtered_on_exist = all_data.where(lambda x: x['exists']).dropna(how='all').reset_index(drop=True)
filtered_on_exist = filtered_on_exist.where(lambda x: x['caption'] != "").dropna(how='all').reset_index(drop=True)
display(filtered_on_exist)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,a close up of a white swan with a black and wh...,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,D:\data\images\itookapicture\thumbnail\b9d9119...,True,False
21395,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,a woman in a black dress holding a skull headd...,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,D:\data\images\itookapicture\thumbnail\17fc708...,True,False
21396,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,a close up of a bald eagle sitting on a tree b...,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,D:\data\images\itookapicture\thumbnail\a7205f6...,True,False
21397,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,there are two flamingos standing in the grass ...,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,D:\data\images\itookapicture\thumbnail\827260f...,True,False


CPU times: total: 312 ms
Wall time: 403 ms


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,a close up of a white swan with a black and wh...,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,D:\data\images\itookapicture\thumbnail\b9d9119...,True,False
21395,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,a woman in a black dress holding a skull headd...,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,D:\data\images\itookapicture\thumbnail\17fc708...,True,False
21396,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,a close up of a bald eagle sitting on a tree b...,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,D:\data\images\itookapicture\thumbnail\a7205f6...,True,False
21397,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,there are two flamingos standing in the grass ...,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,D:\data\images\itookapicture\thumbnail\827260f...,True,False


CPU times: total: 297 ms
Wall time: 360 ms


In [20]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes","trippinthroughtime"]},
	{"name": "SexyDiffusion", "data": ["sfwpetite","selfies","Amicute","amihot","AmIhotAF","HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]}
]
sources_df = pd.DataFrame.from_records(sources)
display(sources_df)

,name,data
0,CityDiffusion,[CityPorn]
1,NatureDiffusion,[EarthPorn]
2,CosmicDiffusion,[spaceporn]
3,ITAPDiffusion,[itookapicture]
4,MemeDiffusion,"[memes, trippinthroughtime]"
5,SexyDiffusion,"[sfwpetite, selfies, Amicute, amihot, AmIhotAF..."
6,FatSquirrelDiffusion,[fatsquirrelhate]
7,RedHeadDiffusion,[SFWRedheads]
8,NextDoorGirlsDiffusion,[SFWNextDoorGirls]


CPU times: total: 31.2 ms
Wall time: 24 ms


,name,data
0,CityDiffusion,[CityPorn]
1,NatureDiffusion,[EarthPorn]
2,CosmicDiffusion,[spaceporn]
3,ITAPDiffusion,[itookapicture]
4,MemeDiffusion,"[memes, trippinthroughtime]"
5,SexyDiffusion,"[sfwpetite, selfies, Amicute, amihot, AmIhotAF..."
6,FatSquirrelDiffusion,[fatsquirrelhate]
7,RedHeadDiffusion,[SFWRedheads]
8,NextDoorGirlsDiffusion,[SFWNextDoorGirls]


CPU times: total: 31.2 ms
Wall time: 30 ms


In [21]:
%%time

smaller_exportable_df = pd.DataFrame(
	data=[filtered_on_exist['path'], filtered_on_exist['image_name'], filtered_on_exist['caption'], filtered_on_exist['title'], filtered_on_exist['subreddit']],
	index=['path', 'image_name', 'caption', 'title', 'subreddit']).T

with ProgressBar():
	smaller_exportable_df['name'] = smaller_exportable_df.progress_apply(lambda x: add_source(x, sources), axis=1)

display(smaller_exportable_df)

global: 100%|██████████| 21399/21399 [00:01<00:00, 12587.68it/s]


,path,image_name,caption,title,subreddit,name
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion
2,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,a man with a beard and a beard sitting in fron...,Anon wants Elon cut,greentext,
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion
...,...,...,...,...,...,...
21394,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion
21395,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion
21396,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion
21397,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion


CPU times: total: 3.28 s
Wall time: 4.36 s


global: 100%|██████████| 21399/21399 [00:01<00:00, 11829.17it/s]


,path,image_name,caption,title,subreddit,name
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion
2,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,a man with a beard and a beard sitting in fron...,Anon wants Elon cut,greentext,
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion
...,...,...,...,...,...,...
21394,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion
21395,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion
21396,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion
21397,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion


CPU times: total: 4.7 s
Wall time: 8.97 s


In [22]:
%%time

with cb:
	dask_frame = dd.from_pandas(smaller_exportable_df, npartitions=10)
	smaller_exportable_df['image_data'] = dask_frame.apply(lambda x: add_compressed_data(x), meta=('str', object), axis=1).compute()

display(smaller_exportable_df)

[                                        ] | 0% Completed | 45.82 ms

global:   0%|          | 0/10 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 16.10 sms[Errno 2] No such file or directory: 'D:\\data\\images\\itookapicture\\71daa45cc13faf381458f972eaba44c3.jpg'
[                                        ] | 0% Completed | 76.77 s[Errno 2] No such file or directory: 'D:\\data\\images\\itookapicture\\5ce0690cecd81e5393bc4d294f942052.jpg'
[########################################] | 100% Completed | 100.09 s


,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,a man with a beard and a beard sitting in fron...,Anon wants Elon cut,greentext,,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
21394,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21395,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21396,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21397,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 48.9 s
Wall time: 1min 43s
[                                        ] | 0% Completed | 3.23 ms

global:   0%|          | 0/10 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 21.90 sms[Errno 2] No such file or directory: 'D:\\data\\images\\itookapicture\\71daa45cc13faf381458f972eaba44c3.jpg'
[                                        ] | 0% Completed | 415.88 s[Errno 2] No such file or directory: 'D:\\data\\images\\itookapicture\\5ce0690cecd81e5393bc4d294f942052.jpg'
[########################################] | 100% Completed | 467.01 s
[########################################] | 100% Completed | 466.75 s


,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,a man with a beard and a beard sitting in fron...,Anon wants Elon cut,greentext,,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
21394,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21395,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21396,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
21397,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 52.3 s
Wall time: 7min 48s


In [23]:
%%time

filtered_again = smaller_exportable_df.where(lambda x: x['image_data'] != "").dropna(how='all').reset_index(drop=True)
filtered_again = filtered_again.where(lambda x: x['name'] != "").dropna(how='all').reset_index(drop=True)
display(filtered_again)

,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,v3ra9g4vrb9a1.jpg,blonde haired woman in a bikini top and bikini...,Braids,sfwpetite,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
19362,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19363,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19364,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19365,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 797 ms
Wall time: 2.91 s


,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,v3ra9g4vrb9a1.jpg,blonde haired woman in a bikini top and bikini...,Braids,sfwpetite,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
19362,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19363,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19364,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19365,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 750 ms
Wall time: 965 ms


In [24]:
%%time

final = filtered_again
display(final)

grouped = final.groupby('name')
groupings = [grouped.get_group(x) for x in grouped.groups]

for group in groupings:
	group.to_parquet(f"data/curated/{group['name'].iloc[0]}.parquet")

,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,v3ra9g4vrb9a1.jpg,blonde haired woman in a bikini top and bikini...,Braids,sfwpetite,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
19362,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19363,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19364,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19365,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 2min 53s
Wall time: 18min 44s


,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,blonde woman with blonde hair and tattoos on h...,Just looking for entertainment,AmIhotAF,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,v3ra9g4vrb9a1.jpg,blonde haired woman in a bikini top and bikini...,Braids,sfwpetite,SexyDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
19362,D:\data\images\itookapicture\b9d9119d89c2473df...,b9d9119d89c2473df5828a15db9f5264.jpg,a close up of a white swan with a black and wh...,ITAP of a swan,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19363,D:\data\images\itookapicture\17fc7087b047d9e5b...,17fc7087b047d9e5b5da9ce299077706.jpg,a woman in a black dress holding a skull headd...,ITAP of a witch,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19364,D:\data\images\itookapicture\a7205f636ae0be5b3...,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,a close up of a bald eagle sitting on a tree b...,ITAP of a Bald Eagle,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
19365,D:\data\images\itookapicture\827260f27293bf219...,827260f27293bf219f260ed275534368.jpg,there are two flamingos standing in the grass ...,ITAP of flamingos.,itookapicture,ITAPDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


CPU times: total: 2min 41s
Wall time: 11min 40s


In [25]:
lines = []
training_lines = []
for record in final.to_dict(orient='records'):
	subreddit = record['subreddit']
	name = record['name']
	prompt = record['title']
	caption = record['caption']
	line = f"<|startoftext|><|model|>{name}<|prompt|>{prompt}<|text|>{caption}<|endoftext|>" + "\n"
	lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))

In [26]:
data_paths = os.listdir("data/curated")

dfs = []

for path in data_paths:
	foo = os.path.join("data", "curated", path)
	df = pd.read_parquet(foo)
	dfs.append(df)

final = pd.concat(dfs)
display(final)

,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
7,D:\data\images\CityPorn\slyqoecbzb9a1.jpg,slyqoecbzb9a1.jpg,arafed view of a pier with a pier and a pier,Pacific Beach California,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
8,D:\data\images\CityPorn\09kuna4hzb9a1.jpg,09kuna4hzb9a1.jpg,mountainside view of a town with a flagpole an...,"Not exactly a city, but Jim Thorpe PA is very ...",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
13,D:\data\images\CityPorn\4y6hzocyec9a1.jpg,4y6hzocyec9a1.jpg,arafed view of a train on a bridge over a river,"Wissahickon, Phila, PA",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
14,D:\data\images\CityPorn\8xwnx4dlgc9a1.jpg,8xwnx4dlgc9a1.jpg,arafed image of a city street scene with peopl...,"2:00am in Nashville, Tennessee",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
15863,D:\data\images\trippinthroughtime\thumbnail\33...,339083838d089fc8fe66c0afea5d7871.jpg,a painting of a man laying on a horse in a stable,"He told me ""Neigh!""",trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15864,D:\data\images\trippinthroughtime\thumbnail\6f...,6f0b42ae45c3c5a1e3285fe23c5d7295.jpg,a picture taken from a picture of a man with a...,"Awww, have a bad dream buddy?",trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15865,D:\data\images\trippinthroughtime\thumbnail\f0...,f040d5fc4198b4ead5a0d275b99533f7.jpg,a picture taken from a book cover of a woman w...,The worst way to start your day,trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15866,D:\data\images\trippinthroughtime\thumbnail\34...,34559f17e64ed6b7933c056b3c3dd635.jpg,a painting of a painting of a man in a gladion,When your rent is too damn high,trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


,path,image_name,caption,title,subreddit,name,image_data
0,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,cars are parked on the side of the road in the...,New York in the fog,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
7,D:\data\images\CityPorn\slyqoecbzb9a1.jpg,slyqoecbzb9a1.jpg,arafed view of a pier with a pier and a pier,Pacific Beach California,CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
8,D:\data\images\CityPorn\09kuna4hzb9a1.jpg,09kuna4hzb9a1.jpg,mountainside view of a town with a flagpole an...,"Not exactly a city, but Jim Thorpe PA is very ...",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
13,D:\data\images\CityPorn\4y6hzocyec9a1.jpg,4y6hzocyec9a1.jpg,arafed view of a train on a bridge over a river,"Wissahickon, Phila, PA",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
14,D:\data\images\CityPorn\8xwnx4dlgc9a1.jpg,8xwnx4dlgc9a1.jpg,arafed image of a city street scene with peopl...,"2:00am in Nashville, Tennessee",CityPorn,CityDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...,...,...,...,...
15863,D:\data\images\trippinthroughtime\thumbnail\33...,339083838d089fc8fe66c0afea5d7871.jpg,a painting of a man laying on a horse in a stable,"He told me ""Neigh!""",trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15864,D:\data\images\trippinthroughtime\thumbnail\6f...,6f0b42ae45c3c5a1e3285fe23c5d7295.jpg,a picture taken from a picture of a man with a...,"Awww, have a bad dream buddy?",trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15865,D:\data\images\trippinthroughtime\thumbnail\f0...,f040d5fc4198b4ead5a0d275b99533f7.jpg,a picture taken from a book cover of a woman w...,The worst way to start your day,trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
15866,D:\data\images\trippinthroughtime\thumbnail\34...,34559f17e64ed6b7933c056b3c3dd635.jpg,a painting of a painting of a man in a gladion,When your rent is too damn high,trippinthroughtime,TrippingThroughTimeDiffusion,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
